# The fixed `tasks.yaml` file

## What is `tasks.yaml`?

The `tasks.yaml` file describes:

* The **steps (tasks)** your CrewAI agents will perform.
* What **each task expects as input** and must produce as output.
* Which **agent is responsible** for doing each task.
* How tasks are **linked together** via context.

Each task maps directly to the role of one of your agents from `agents.yaml`.

<br><br>
## `filter_emails`

#### **What it does**

```yaml
description: >
  Analyze the following list of unread emails and assign each one to a category...
```

* This is the **first task** in the pipeline.
* It takes in a batch of **unread emails** and classifies them into four categories:

  * `"Urgent"`
  * `"Informational"`
  * `"Spam"`
  * `"Action Required"`

#### **Important constraints**

```yaml
🛑 Do not modify the original structure. Preserve the `id` and `threadId` exactly as provided.
```

* The LLM is **explicitly warned not to alter IDs**, which are critical for threading and tracking.
* This instruction ensures safe and consistent automation.

#### **Expected Output**

```yaml
expected_output: >
  A list of categorized emails, each with id, threadId, category, and justification.
```

* A structured JSON-like list.
* Includes both classification and reasoning (`justification`), which is useful for debugging and transparency.

#### **Agent**

```yaml
agent: email_filter_agent
```

* The filtering logic is delegated to the **`email_filter_agent`** from `agents.yaml`.

#### ❌ Removed Context

```yaml
# ❌ Removed: context: [emails]
```

* Emails are passed directly as input, not via a named prior task.
* This makes it the **starting point** of the workflow.


<br><br>
## `action_required_emails`

#### **What it does**

```yaml
description: >
  Review the emails marked as "Action Required" and determine the exact nature of the required action...
```

* This task refines the `"Action Required"` emails by deciding **what kind of action** is needed:

  * Reply
  * Schedule meeting
  * Research
  * Delegate
  * etc.

#### **Expected Output**

```yaml
expected_output: >
  A list of actionable emails, each with:
  - `id`
  - `threadId`
  - `action`
  - `reason`
```

* The structure is well defined and prepares for the **next step** (reply drafting).
* `reason` gives a rationale, which can be helpful for logging or future improvements.

#### **Agent**

```yaml
agent: email_action_agent
```

* Uses the **`email_action_agent`** to evaluate tasks.

#### **Context**

```yaml
context:
  - filter_emails
```

* This task **depends on the output** of the `filter_emails` task.
* CrewAI passes the filtered email data into this task automatically.



<br><br>
## `draft_responses`

#### **What it does**

```yaml
description: >
  Based on the list of actionable emails, draft appropriate responses...
```

* This task drafts actual **reply messages**.
* It focuses on professionalism, brevity, and relevance.

#### **Expected Output**

```yaml
expected_output: >
  A Markdown-formatted set of email reply drafts...
```

* The output is human-readable and can be previewed before sending.
* It groups each draft by email subject.

#### **Agent**

```yaml
agent: email_response_writer
```

* Uses the **`email_response_writer`** for generating the drafts.

#### **Context**

```yaml
context:
  - action_required_emails
```

* It builds directly on the previous step's filtered action items.


<br><br>
## Key Points

**Task Chaining via Context**
* Tasks pass data to one another in sequence using the `context` keyword.
* This creates a clear, readable pipeline.

**Strict Output Formats**
* Each task defines exactly what kind of output is expected — this is crucial for the next task to work correctly.

**Use of Constraints**
* Instructions like “Do not modify `threadId`” prevent errors and ensure system stability.

**Agent-to-Task Mapping**
* Each task clearly maps to a single agent — reinforcing separation of responsibilities.

**Markdown Output for UI**
* Final drafts are Markdown-formatted to support easy rendering in notebooks, UIs, or email editors.